In [ ]:
#pip install pandas

In [ ]:
from platform import python_version
print(python_version())
import pandas as pd 
import numpy as np
import gc

In [ ]:
Dataset_folder = "./ml-25m"

In [ ]:
#movies = pd.read_csv(f"{Dataset_folder}/movies.csv")
#ratings = pd.read_csv(f"{Dataset_folder}/ratings.csv", chunksize=1000, usecols = ['userId','movieId', 'rating'])
#tags = pd.read_csv(f"{Dataset_folder}/tags.csv", usecols = ['userId','movieId', 'tag'])

In [ ]:
#converts each users reviews to a list of tuples including the movieID and the 
def extract_users (fn):
    users = {}
    count = 0 
    header = True
    with pd.read_csv(fn, chunksize=10000) as reader:
        for chunk in reader:
            for index, row in chunk.iterrows():
                count +=1 
                if (row['userId'] not in users.keys()):
                    users [row['userId']] = [(row['movieId'], row['rating'])]
                else:
                    users [row['userId']].append((row['movieId'], row['rating']))
                if (count % 100000 == 0 ):
                    tempdict = {"userId":list(users.keys())[:-1], "ratings_list":list(users.values())[:-1]}
                    #print (tempdict)
                    tempDF = pd.DataFrame.from_dict(tempdict)

                    tempDF.to_csv("ratings_organized.csv",header=header, mode='a')

                    header = False
                    
                    for i, key in enumerate(list(users.keys())[:-1]):
                        del users [key]
                    del tempDF
                    gc.collect()
                    
    
    return users

            
    
    

In [ ]:
extract_users(f"{Dataset_folder}/ratings.csv")

In [ ]:
#organized_reviews.to_csv("organized_reviews.csv", index=False, chunksize=10000 encoding='utf-8')

## Mesuring user simillarity:

### Case 1: Number of overlapping movies watched 

- going on a point based system we will add 100 points in the simillarity metric for every movie that both users have watched
- we must later account for movies that they both watched but had different opinions of 

### Case 2: rating similarity for movies both users have watched

- to properly mesure simillarity in taste we must also account for differences in opinions on movies two users have both watched 
- So we will subtract points based on a percentage of difference between the two users
    - simillarity = simillarity - 100* (1-(rating user1/rating user 2))
```
Ex. 
movie: shawshank redemption
user 1 rating: 1
user 2 rating: 5 
Case 1 :similarity = 100 for sharing the same film, 
Case 2: similarity = 100 - 100 *(1 - (1/5))  = 100 - 100(0.8) = 100 - 80 = 20 
new simillarity is now 20 after it has been balenced for ratings
```

### Case 3:  overall genre prefrences of the users

- We should also take into account general genre preferences as a mesure of simillarity. 
- we will also consider genre simillarity as a dividend of the percentage of each genre the user has watched 
- if 1000 points is the total afforded to each genre, simillarity will be incrmented by the following expression
- for each genre
    - similarity += 1000 * (precent of movies in certain genre for user 1/ percent for user 2)* the smaller percent from the two   users
    
    
## final step:
#### we will normalize all simillarity values between 0 and 1 using z normalization


In [1]:
from platform import python_version
print(python_version())
import pandas as pd 
import numpy as np
import gc

3.8.1


In [2]:
Dataset_folder = "./ml-25m"

In [3]:
Userratings = pd.read_csv(f"./ratings_organized.csv", usecols = ['userId', 'ratings_list'])

In [4]:
#Userratings

In [5]:
from pickle import dump,load

In [6]:
movies = pd.read_csv(f"{Dataset_folder}/movies.csv",usecols = ['movieId','genres'])

In [7]:
## only need to do this once 
# "banana|apple".split("|")
# genres = []
# for index, row in movies.iterrows():
#     gs = row['genres'].split("|")
#     for g in gs:
#         if (g not in genres):
#             genres.append(g)
            
# print(len(genres))
# dump(genres, open("genresList.pkl","wb"))


In [8]:
file = open("genresList.pkl",'rb')
genres = load(file)
file.close()

In [9]:
def create_genre_tracker ():
    gt = {}
    for g in genres:
        gt[g] = 0 
    return gt


In [10]:
result =movies.loc[movies['movieId'] == 1 ,'genres'].values[0]
print(result)

Adventure|Animation|Children|Comedy|Fantasy


In [11]:
import ast
def compute_simillarity_ratings (l1,l2):
    l1_dict = dict(l1)
    l2_dict = dict(l2)
    simillarity = 0 
    
    genre_tracker1 = create_genre_tracker ()
    genre_tracker2 = create_genre_tracker ()
    
    l2_keys = list(l2_dict.keys())
    for key in l1_dict.keys():
        m1_genres = movies.loc[movies['movieId'] == key  ,'genres'].values[0].split("|")
        for mg in m1_genres:
            genre_tracker1 [mg] +=1
        if key in l2_keys:
            r1 = l1_dict [key]
            r2 = l2_dict [key]
            if (r1 < r2):
                simillarity += 100*(r1/r2)
            else:
                simillarity+= 100*(r2/r1) 
                
    for key in l2_keys:
        m2_genres = movies.loc[movies['movieId'] == key  ,'genres'].values[0].split("|")
        for mg in m2_genres:
            genre_tracker2 [mg] +=1
            
    genre_simillarity =  0 
    for g in genres:
        tempg1 = genre_tracker1 [g]
        tempg2 = genre_tracker2 [g]
        if (tempg1 > 0 or tempg2 >0):
            genre_simillarity += (1000* (min(tempg1,tempg2)/max(tempg1,tempg2))* min (tempg1,tempg2))
        
    return genre_simillarity + simillarity
                
        
        

In [12]:
num_users = len(Userratings.sample(n=100000).index)
del Userratings
gc.collect()
print(num_users)

100000


In [13]:
#converts each users reviews to a list of tuples including the movieID and the 
def save_simillarity (fn,numsamples):
    user_similarity = {}
    df = pd.read_csv(fn).sample(n=numsamples)
    header = True
    for index, row in df.iterrows():
        for i in range (0,len(df.index)):
            rev1 = ast.literal_eval (row["ratings_list"])
            rev2 = ast.literal_eval(df.iloc[[i]] ["ratings_list"].tolist()[0])
            if (row['userId'] not in user_similarity.keys()):
                user_similarity [row['userId']] = [compute_simillarity_ratings(rev1,rev2)]
            else:
                user_similarity [row['userId']].append(compute_simillarity_ratings(rev1,rev2))

        if (index % 100 == 0 or index == len(df.index)-1 ):
            print(index)
            tempdict = {"userId":list(user_similarity.keys())[:-1], "simillarity_vector":list(user_similarity.values())[:-1]}
            #print (tempdict)
            tempDF = pd.DataFrame.from_dict(tempdict)

            tempDF.to_csv("simillarity_matrix.csv",header=header, mode='a')

            header = False

            for i, key in enumerate(list(user_similarity.keys())[:-1]):
                del user_similarity [key]
            del tempDF
            gc.collect()

    
    return

In [ ]:
save_simillarity(f"./ratings_organized.csv",100000)

In [ ]:
print(type([1,2,3]))